In [1]:
import os.path as op
import matplotlib.pyplot as plt
from itertools import compress
import time
import logging
import mne
from mne import events_from_annotations
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from itertools import compress
import mne
import mne_nirs
from mne.preprocessing.nirs import optical_density, beer_lambert_law

from mne_nirs.signal_enhancement import enhance_negative_correlation, short_channel_regression

from mne.preprocessing.nirs import (optical_density,
                                    temporal_derivative_distribution_repair,
                                    scalp_coupling_index)

import ROI
from ROI import different_hb, different_roi
from meta import *
from filter_params import FILTER_DICT
from functions_fnirs import *

def fast_scanfiles(dirname, contains=None):
    subfiles = [f.path for f in os.scandir(dirname) if f.is_file()]
    if contains != None:
        subfiles = [i for i in subfiles if contains in i ]
    return subfiles

In [2]:
from matplotlib.colors import LinearSegmentedColormap

fnirs_colors = dict(
    hbo='#C91111',
    hbt= '#5B324B',
    hbr='#004E7C',
  )

legend_params_dict = dict(
                            loc='center', 
                            fontsize=11,       
                            borderpad=1.5, 
                            labelspacing=1.5,
                            markerscale=10, 
                            framealpha=0
)

curves_subplot_params = dict(
 top=0.92,
 bottom=0.11,
 left=0.065,
 right=0.945,
 hspace=0.2,
 wspace=0.35
)

two_topomaps_subplot_params = dict(top=0.89,
bottom=0.21,
left=0.115,
right=0.98,
hspace=0.175,
wspace=0.0)

one_topomap_subplot_params = dict(top=0.92,
bottom=0.075,
left=0.105,
right=0.98,
hspace=0.105,
wspace=0.0)


mask_params = dict(marker='o', 
                   markerfacecolor='black', 
                   markeredgecolor='white',
                   linewidth=0, 
                   markersize=6
                   )

color_list = [(0, fnirs_colors['hbr']), (0.5, '#FFFFE4'), (1, fnirs_colors['hbo'])]
custom_cmap = LinearSegmentedColormap.from_list('Custom_cmap', color_list, )

In [78]:
def make_subdicts(haemo_dict, contains):
    new_dict = {key[3:]:value for key, value in haemo_dict.items() if contains in key}
    return new_dict


def concatenate_dicts(haemo_dict, new_shape=(20, 32, 30)):
    new_dict = {}
    for key in haemo_dict.keys():
        a = np.vstack(haemo_dict[key])
        if new_shape:
            a = a.reshape(new_shape[0], new_shape[1], new_shape[2])
        new_dict[key] = a
    return new_dict

def get_roi_evokeds(new_dict, key, slicer=[6, 13]):
    M1 = new_dict[key].mean(axis=0)[:slicer[0], :].mean(axis=0)
    S1 = new_dict[key].mean(axis=0)[slicer[0]:slicer[1], :].mean(axis=0)
    SMZ = new_dict[key].mean(axis=0)[slicer[1]:, :].mean(axis=0)
    return M1, S1, SMZ

In [4]:
path_to_haemo = 'haemodynamics_np'

haemo_dict = dict(
    
ME_smr_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='ME_smr_left')],
MI_smr_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='MI_smr_left')],
TS_smr_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='TS_smr_left')],
TI_smr_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='TI_smr_left')],
SA_smr_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='SA_smr_left')],

ME_rest_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='ME_rest_left')],
MI_rest_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='MI_rest_left')],
TS_rest_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='TS_rest_left')],
TI_rest_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='TI_rest_left')],
SA_rest_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='SA_rest_left')],


ME_smr_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='ME_smr_right')],
MI_smr_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='MI_smr_right')],
TS_smr_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='TS_smr_right')],
TI_smr_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='TI_smr_right')],
SA_smr_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='SA_smr_right')],

ME_rest_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='ME_rest_right')],
MI_rest_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='MI_rest_right')],
TS_rest_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='TS_rest_right')],
TI_rest_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='TI_rest_right')],
SA_rest_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='SA_rest_right')],
)



ME_subdict = make_subdicts(haemo_dict, contains='ME')
MI_subdict = make_subdicts(haemo_dict, contains='MI')
TS_subdict = make_subdicts(haemo_dict, contains='TS')
TI_subdict = make_subdicts(haemo_dict, contains='TI')
SA_subdict = make_subdicts(haemo_dict, contains='SA')

In [5]:


ME_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='ME'))
MI_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='MI'))
TS_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='TS'))
TI_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='TI'))
SA_subdict = concatenate_dicts(make_subdicts(haemo_dict, contains='SA'))

In [8]:
%matplotlib auto
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

Using matplotlib backend: QtAgg


In [19]:
info_hbr_total = mne.io.read_info("/mnt/diskus/infos/info_hbr_total_info.fif")
info_left_smz = mne.io.read_info("/mnt/diskus/infos/info_left_smz_info.fif")
info_right_smz = mne.io.read_info("/mnt/diskus/infos/info_right_smz_info.fif")
info_hbo_total = mne.io.read_info("/mnt/diskus/infos/info_hbo_total_info.fif")

In [24]:
curves_hb = 'hbo'
SUBJECT = 'AVERAGE'
CONDITION = 'SA'
subdict = SA_subdict


M1_evoked_SMR_left, S1_evoked_SMR_left, SMZ_evoked_SMR_left = get_roi_evokeds(subdict, key='smr_left')
M1_evoked_REST_left, S1_evoked_REST_left, SMZ_evoked_REST_left = get_roi_evokeds(subdict, key='rest_left')
M1_evoked_SMR_right, S1_evoked_SMR_right, SMZ_evoked_SMR_right = get_roi_evokeds(subdict, key='smr_right')
M1_evoked_REST_right, S1_evoked_REST_right, SMZ_evoked_REST_right = get_roi_evokeds(subdict, key='rest_right')


In [25]:
### PLOT HEMODYNAMICS PER SE ###
m1_group_left = [info_left_smz.ch_names.index(ch) for ch in different_roi['M1'][0] ]
s1_group_left = [info_left_smz.ch_names.index(ch) for ch in different_roi['S1'][0] ]
smz_group_left = [info_left_smz.ch_names.index(ch) for ch in different_roi['REMAINING'][0]]

m1_group_right = [info_right_smz.ch_names.index(ch) for ch in different_roi['M1'][2] ]
s1_group_right = [info_right_smz.ch_names.index(ch) for ch in different_roi['S1'][2] ]
smz_group_right = [info_right_smz.ch_names.index(ch) for ch in different_roi['REMAINING'][2]]

times = np.arange(TMIN, TMAX, 1/2)
linewidth = 1.5
ylims = (-1000, 1500)
tmin, tmax = TMIN, TMAX
topo_linewidth = 1
pointsize = 20
topo_width = topo_height = '30%'

fig, axes = plt.subplots(1, 2, figsize=(20, 12))

ylims=(-12, 16)


### LEFT PART ###

line_m1_smr, = axes[0].plot(times, M1_evoked_SMR_left*1e6, label=f'M1/{curves_hb} SMR', 
         color=fnirs_colors['hbr'])
line_s1_smr, = axes[0].plot(times, S1_evoked_SMR_left*1e6, label=f'S1/{curves_hb} SMR', 
         color=fnirs_colors['hbo'])
line_smz_smr, = axes[0].plot(times, SMZ_evoked_SMR_left*1e6, label=f'SMR/{curves_hb} SMR', 
         color=fnirs_colors['hbt'])
line_m1_rest, = axes[0].plot(times, M1_evoked_REST_left*1e6, label=f'M1/{curves_hb} REST', 
         color=fnirs_colors['hbr'], linestyle='--')
line_s1_rest, = axes[0].plot(times, S1_evoked_REST_left*1e6, label=f'S1/{curves_hb} REST', 
         color=fnirs_colors['hbo'], linestyle='--')
line_smz_rest, = axes[0].plot(times, SMZ_evoked_REST_left*1e6, label=f'SMR/{curves_hb} REST', 
         color=fnirs_colors['hbt'], linestyle='--')

fill_1 = filler_between(axes[0], ylims)

set_axis_properties(axes[0],
                    ylims=ylims, 
                    tlims=(tmin, tmax), 
                    title=f'HbO response in LEFT hemisphere\n {SUBJECT} in 20 subjects, Condition {CONDITION}',
                    xlabel='Time, s', 
                    ylabel='Hb concentration, Δ μM\L',  
                    linewidth=1.5, 
                    fontsize=14, 
                    title_size=18,
                    legend_flag=False
                        )
### LEFT HEMO TOPOMAP ###
inset_ax1 = inset_axes(axes[0], width=topo_width, height=topo_height, loc="lower right")
mne.viz.plot_sensors(info_left_smz, 
                     ch_groups=[  m1_group_left, 
                                 smz_group_left,
                                s1_group_left],
                     axes=inset_ax1,
                     pointsize=pointsize, 
                     linewidth=topo_linewidth)

### RIGHT PART ###
axes[1].plot(times, M1_evoked_SMR_right*1e6, label=f'M1/{curves_hb} SMR', 
         color=fnirs_colors['hbr'])
axes[1].plot(times, S1_evoked_SMR_right*1e6, label=f'S1/{curves_hb} SMR', 
         color=fnirs_colors['hbo'])
axes[1].plot(times, SMZ_evoked_SMR_right*1e6, label=f'SMR/{curves_hb} SMR', 
         color=fnirs_colors['hbt'])
axes[1].plot(times, M1_evoked_REST_right*1e6, label=f'M1/{curves_hb} REST', 
         color=fnirs_colors['hbr'], linestyle='--')
axes[1].plot(times, S1_evoked_REST_right*1e6, label=f'S1/{curves_hb} REST', 
         color=fnirs_colors['hbo'], linestyle='--')
axes[1].plot(times, SMZ_evoked_REST_right*1e6, label=f'SMR/{curves_hb} REST', 
         color=fnirs_colors['hbt'], linestyle='--')

fill_2 = filler_between(axes[1], ylims)


set_axis_properties(axes[1],
                    ylims=ylims, 
                    tlims=(tmin, tmax), 
                    title=f'HbO response in RIGHT hemisphere\n {SUBJECT} in 20 subjects, Condition {CONDITION}',
                    xlabel='Time, s', 
                    ylabel='',  
                    linewidth=1.5, 
                    fontsize=14, 
                    title_size=18,
                    legend_flag=False,
                        )

### RIGHT HEMO TOPOMAP ###
inset_ax2 = inset_axes(axes[1], width=topo_width, height=topo_height, loc="lower right")
mne.viz.plot_sensors(
                    info_right_smz, 
                    ch_groups=[
                                m1_group_right, 
                                smz_group_right,
                                s1_group_right],
                    axes=inset_ax2, 
                    pointsize=pointsize, 
                    linewidth=topo_linewidth)
                     

### LEGENDS PARAMS ###
ax_legend = fig.add_axes([0.25, 0.25, 0.5, 0.25])
ax_legend.legend([line_m1_smr, line_s1_smr, line_smz_smr,
                 line_m1_rest, line_s1_rest, line_smz_rest, fill_1], 
                 [f'M1/{curves_hb} SMR', f'S1/{curves_hb} SMR', f'SMZ/{curves_hb} SMR', 
                  f'M1/{curves_hb} REST', f'S1/{curves_hb} REST',f'SMZ/{curves_hb} REST',
                 'Task duration'], 
                **legend_params_dict) 

ax_legend.set_frame_on(False)
ax_legend.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

plt.subplots_adjust(**curves_subplot_params)




Relation plotting

In [ ]:
path_to_haemo = '/mnt/diskus/fnirs-data-analysis-git/relation_np/'

haemo_rel_dict = dict(
    
ME_rel_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='ME_rel_left')],
MI_rel_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='MI_rel_left')],
TS_rel_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='TS_rel_left')],
TI_rel_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='TI_rel_left')],
SA_rel_left = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='SA_rel_left')],



ME_rel_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='ME_rel_right')],
MI_rel_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='MI_rel_right')],
TS_rel_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='TS_rel_right')],
TI_rel_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='TI_rel_right')],
SA_rel_right = [np.load(i) for i in fast_scanfiles(path_to_haemo,
                                        contains='SA_rel_right')],

)


ME_rel_subdict = concatenate_dicts(make_subdicts(haemo_rel_dict, contains='ME'), new_shape=(20, 3, 30))
MI_rel_subdict = concatenate_dicts(make_subdicts(haemo_rel_dict, contains='MI'), new_shape=(20, 3, 30))
TS_rel_subdict = concatenate_dicts(make_subdicts(haemo_rel_dict, contains='TS'), new_shape=(20, 3, 30))
TI_rel_subdict = concatenate_dicts(make_subdicts(haemo_rel_dict, contains='TI'), new_shape=(20, 3, 30))
SA_rel_subdict = concatenate_dicts(make_subdicts(haemo_rel_dict, contains='SA'), new_shape=(20, 3, 30))

In [67]:
curves_hb = 'hbo'
SUBJECT = 'AVERAGE'
CONDITION = 'SA'
subdict = SA_rel_subdict

relation_M1_left, relation_S1_left, relation_SMZ_left = get_roi_evokeds(subdict, key='rel_left', slicer=[1, 2])
relation_M1_right, relation_S1_right, relation_SMZ_right = get_roi_evokeds(subdict, key='rel_right', slicer=[1, 2])

In [69]:
fig, axes = plt.subplots(1, 2, figsize=(20, 12))
times = np.arange(TMIN, TMAX, 1/2)
linewidth = 1.5
ylims=(-3000, 5500)
tmin, tmax = TMIN, TMAX
topo_linewidth = 1
pointsize = 20
topo_width = topo_height = '30%'



### RELATION RIGHT ####

rel_line_m1_smr, = axes[0].plot(times, relation_M1_left, label=f'M1/{curves_hb} SMR', 
        color=fnirs_colors['hbr'])
rel_line_s1_smr, = axes[0].plot(times, relation_S1_left, label=f'S1/{curves_hb} SMR', 
        color=fnirs_colors['hbo'])
rel_line_smz_smr, = axes[0].plot(times, relation_SMZ_left, label=f'SMR/{curves_hb} SMR', 
        color=fnirs_colors['hbt'])

fill_1 = filler_between(axes[0], ylims)

set_axis_properties(
                        axes[0],
                        ylims=ylims, 
                        tlims=(tmin, tmax), 
                        title=f'HbO relation in LEFT hemisphere\n{SUBJECT} in 20 subjects, Condition {CONDITION}',
                        xlabel='Time, s', 
                        ylabel='Hb relation SMR/REST, %',  
                        linewidth=1.5, 
                        fontsize=14, 
                        title_size=18,
                        legend_flag=False
                        )

### TOPO FOR LEFT RELATION PICTURE ###
inset_ax1 = inset_axes(axes[0], 
                        width=topo_width, 
                        height=topo_height, 
                        loc="lower right")

mne.viz.plot_sensors(
                        info_left_smz, 
                        ch_groups=[
                                        m1_group_left, 
                                        smz_group_left,
                                        s1_group_left
                        ],
                        axes=inset_ax1,
                        pointsize=pointsize, 
                        linewidth=topo_linewidth
)

### RELATION RIGHT ###

axes[1].plot(times, relation_M1_right, label=f'M1/{curves_hb} SMR', 
        color=fnirs_colors['hbr'])
axes[1].plot(times, relation_S1_right, label=f'S1/{curves_hb} SMR', 
        color=fnirs_colors['hbo'])
axes[1].plot(times, relation_SMZ_right, label=f'SMR/{curves_hb} SMR', 
        color=fnirs_colors['hbt'])


fill_1 = filler_between(axes[1], ylims)

set_axis_properties(
                        axes[1],
                        ylims=ylims, 
                        tlims=(tmin, tmax), 
                        title=f'HbO relation in RIGHT hemisphere\n{SUBJECT} in 20 subjects, Condition {CONDITION}',
                        xlabel='Time, s', 
                        ylabel='',  
                        linewidth=1.5, 
                        fontsize=14, 
                        title_size=18,
                        legend_flag=False
)

### TOPO FOR RIGHT RELATION PICTURE ###
inset_ax2 = inset_axes(axes[1], width=topo_width, height=topo_height, loc="lower right")
mne.viz.plot_sensors(info_right_smz, 
                        ch_groups=[m1_group_right, 
                                smz_group_right,
                                s1_group_right],
                        axes=inset_ax2, 
                        pointsize=pointsize, 
                        linewidth=topo_linewidth)

### LEGEND PARAMS ###
ax_legend = fig.add_axes([0.25, 0.25, 0.5, 0.25])
ax_legend.legend([rel_line_m1_smr, rel_line_s1_smr, rel_line_smz_smr, fill_1], 
                [f'M1 {curves_hb} relation', 
                f'S1 {curves_hb} relation', 
                f'SMZ {curves_hb} relation', 
                'Task duration'], 
                **legend_params_dict) 

ax_legend.set_frame_on(False)
ax_legend.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.subplots_adjust(**curves_subplot_params)



Plot rest and smr topomaps

In [90]:
topo_dir = '/mnt/diskus/fnirs-data-analysis-git/topomaps_np'
topo_dict = dict(
    
ME_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='ME')],
MI_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='MI')],
TS_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='TS')],
TI_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='TI')],
SA_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='SA')],
)
huh = concatenate_dicts(topo_dict, new_shape=None)

In [116]:
curves_hb = 'hbo'
SUBJECT = 'AVERAGE'
CONDITION = 'TS'
subdict = huh['TS_topo']
mean_topos = subdict.mean(axis=0)

In [117]:
smr_in_peak_timestamp = mean_topos[:102]
rest_in_peak_timestamp = mean_topos[102:]

In [118]:
top_n_chans = 10
mask_SMR, top_dict_SMR = get_top_channels_mask(smr_in_peak_timestamp, 
                                                info_hbo_total, 
                                                top_n_chans)
mask_REST, top_dict_REST = get_top_channels_mask(rest_in_peak_timestamp, 
                                                info_hbo_total, 
                                                top_n_chans)



ylims = (-6, 8)

fig, axes = plt.subplots(1, 2, figsize=(20, 12))

### LEFT SMR TOPO ###
a = mne.viz.plot_topomap(data=smr_in_peak_timestamp,
                        pos=info_hbo_total,
                        axes=axes[0],
                        vlim=ylims,
                        contours=6,
                        extrapolate='local',
                        image_interp='linear',
                        cmap=custom_cmap, 
                        mask=mask_SMR,
                        mask_params=mask_params,
                        show=False)
### RIGHT REST TOPO ###
a = mne.viz.plot_topomap(data=rest_in_peak_timestamp,
                        pos=info_hbo_total,
                        axes=axes[1],
                        vlim=ylims,
                        contours=6,
                        extrapolate='local',
                        image_interp='linear',
                        cmap=custom_cmap,
                        mask=mask_REST,
                        mask_params=mask_params, 
                        show=False)

### TITLES ###
axes[0].set_title(
        f'Topography of {curves_hb} in {SUBJECT}, 20 subjects\n in {CONDITION} condititon',
fontsize=18)

axes[1].set_title(
        f'Topography of {curves_hb} in {SUBJECT}, 20 subjects \n in REST condititon',
fontsize=18)

### COLORBAR SETTINGS ###
sm = plt.cm.ScalarMappable(cmap=custom_cmap, 
                        norm=matplotlib.colors.Normalize(vmin=ylims[0], vmax=ylims[1]))

cbaxes = fig.add_axes([0.075, 0.25, 0.02, 0.5]) # setup colorbar axes. 
cbar = plt.colorbar(mappable=sm, cax=cbaxes, pad=0.15, orientation='vertical')
cbar.set_label(' Hb concentration, Δ μM\L', loc='center', size=12)
cbar.ax.yaxis.set_label_coords(-0.5, 0.5)
plt.subplots_adjust(**two_topomaps_subplot_params)

Plot rel topomaps

In [131]:
topo_dir = '/mnt/diskus/fnirs-data-analysis-git/topomaps_rel_np'
topo_dict = dict(
    
ME_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='ME')],
MI_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='MI')],
TS_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='TS')],
TI_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='TI')],
SA_topo = [np.load(i) for i in fast_scanfiles(topo_dir,
                                        contains='SA')],
)
huh = concatenate_dicts(topo_dict, new_shape=None)



In [146]:
curves_hb = 'hbo'
SUBJECT = 'AVERAGE'
CONDITION = 'SA'
subdict = huh['SA_topo']
mean_topos = subdict.mean(axis=0)

In [147]:
rel_smr_in_peak_timestamp = mean_topos

In [148]:
fig, ax = plt.subplots(figsize=(10, 8))
top_n_chans = 10
rel_mask_SMR, rel_top_dict_SMR = get_top_channels_mask(rel_smr_in_peak_timestamp, 
                                                info_hbo_total, 
                                                top_n_chans)


# min1, max1 = min(rel_smr_in_peak_timestamp), max(rel_smr_in_peak_timestamp)
# ylims = (min(rel_smr_in_peak_timestamp), max(rel_smr_in_peak_timestamp))
ylims = (-5000, 5000)

### LEFT SMR TOPO ###
a = mne.viz.plot_topomap(
                        data=rel_smr_in_peak_timestamp,
                        pos=info_hbo_total,
                        vlim=ylims,
                        axes=ax,
                        contours=6,
                        extrapolate='local',
                        image_interp='linear',
                        cmap=custom_cmap,
                        mask=rel_mask_SMR,
                        mask_params=mask_params,
                        show=False
)


### TITLES ###
ax.set_title(
        f'Topography of relation of {curves_hb} {SUBJECT} in 20 subjects\n in {CONDITION} condititon',
fontsize=18)


### COLORBAR SETTINGS ###
sm = plt.cm.ScalarMappable(cmap=custom_cmap, 
                        norm=matplotlib.colors.Normalize(vmin=ylims[0], vmax=ylims[1]))

cbaxes = fig.add_axes([0.075, 0.25, 0.03, 0.5]) # setup colorbar axes. 
cbar = plt.colorbar(mappable=sm, cax=cbaxes, pad=0.15, orientation='vertical')
cbar.set_label(' Hb relation, %', loc='center', size=12)
cbar.ax.yaxis.set_label_coords(-0.65, 0.5)
plt.subplots_adjust(**one_topomap_subplot_params)